In [9]:
import arcpy
import pandas as pd
import arcpy

## Set Filepaths

In [10]:
arcpy.env.workspace = 'D:/GitHub/UDOT-Master-Segments/data/1_WF/Segments_WF_20231101/'

wf_seg_shapefile = r'..\data\1_WF\Segments_WF_20231101_PREPROCESSED\Segments_WF - 2023-11-01.shp'
subarea_shapefile = r'..\data\Subarea\Subarea.shp'

output_centroids = r'Centroids.shp'
output_subarea_centroids = r'Centroids_with_Subarea.shp'
output_wf_centroids = r'Centroids_within_Subarea1.shp'
output_wf_segments = r'Segments_within_Subarea1.shp'
output_wf_final_segments = r'Segments_WF_20231101.shp'

## List of Duplicate SEGIDs

In [11]:
fields = ["SEGID"]
df = pd.DataFrame([row for row in arcpy.da.SearchCursor(wf_seg_shapefile, fields)])
df.columns = fields  # Set column names

# Identify duplicates based on the SEGID field
duplicates = df[df.duplicated(subset=["SEGID"], keep=False)]

# Display the duplicates
print(duplicates)

Empty DataFrame
Columns: [SEGID]
Index: []


## Clip to WF Subarea

In [12]:
try:
    # Perform FeatureToPoint to create centroids
    arcpy.management.FeatureToPoint(wf_seg_shapefile, output_centroids, "INSIDE")
    # Perform spatial join with centroids and subarea shapefile
    arcpy.analysis.SpatialJoin(target_features=output_centroids, join_features=subarea_shapefile, out_feature_class=output_subarea_centroids, join_type="KEEP_COMMON", match_option="WITHIN")
    
     # Select centroids with SUBAREAID = 1
    sql_expression = "SUBAREAID = 1"
    arcpy.analysis.Select(output_subarea_centroids, output_wf_centroids, sql_expression)
    
    # Perform spatial join with clipped centroids and segments
    arcpy.analysis.SpatialJoin(target_features=wf_seg_shapefile, join_features=output_wf_centroids, out_feature_class=output_wf_segments, join_type="KEEP_COMMON", match_option="INTERSECT")
    
    # Delete fields with names containing "_1"
    field_names = [field.name for field in arcpy.ListFields(output_wf_segments)]
    fields_to_delete = [field for field in field_names if "_1" in field]
    arcpy.management.DeleteField(output_wf_segments, fields_to_delete)

    # Delete the intermediate files
    arcpy.management.Delete(output_centroids)
    arcpy.management.Delete(output_subarea_centroids)
    arcpy.management.Delete(output_wf_centroids)
    
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except Exception as e:
    print(str(e))
    
arcpy.management.Rename(output_wf_segments, output_wf_final_segments)

<Result 'D:/GitHub/UDOT-Master-Segments/data/1_WF/Segments_WF_20231101\\Segments_WF_20231101.shp'>